In [2]:
import numpy as np
import sklearn.decomposition
import imblearn
from pathlib import Path
import tqdm
from datetime import datetime
from massimal import hyspec_io, annotation, hyspec_ml
import json
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'hyspec_io'

In [ ]:
# Paths
hyspec_base = Path('/home/mha114/data/massimal/vega_sola/hyperspec/images/2a_rrs/')
class_json = Path('/home/mha114/data/massimal/vega_sola/hyperspec/annotation/v2.1/png_gray/label_classes.json')
class_hierarchy_json = Path("/home/mha114/code/github/massimal/annotation/massimal_annotation_class_hierarchy.json")
pca_dir = Path('/home/mha114/data/massimal/vega_sola/hyperspec/pca_models')
spectra_file = Path('/home/mha114/data/massimal/vega_sola/hyperspec/sampled_spectra/sola_sampled_spectra_v1/20220823_Vega_Sola_ExtractedSpectraAndLabels2024-04-27-18h25m55s.npz')

timestamp = datetime.now().strftime('%Y-%m-%d-%Hh%Mm%Ss')
pca_model_path = pca_dir / f'vega_sola_pca_model_{timestamp}.npz'


In [ ]:
# Parameters
wl_max = 730
wl_min = 400
minimum_reflectance = 0.001

In [ ]:
# Get wavelengths from example hyspec file
example_hyspec_file_path = [path for path in sorted(hyspec_base.rglob('*.hdr'))][0]
_,wl,_,_ = hyspec_io.load_envi_image(str(example_hyspec_file_path),rgb_only=True)

In [ ]:
# Load data
with np.load(spectra_file) as npz_files:
    X_raw = npz_files['X']
    y_raw = npz_files['y']

In [ ]:
# Remove zero-valued spectra (caused by saturation)
nonzero_spectra_mask = ~np.all(X_raw==0,axis=1)
X = X_raw[nonzero_spectra_mask,:]
y = y_raw[nonzero_spectra_mask]

In [ ]:
# Ensure non-zero reflectances (better behavior when taking logarithm)
X[X<minimum_reflectance] = minimum_reflectance

In [ ]:
# Load class names and indices
class_indices = annotation.read_hasty_metadata(class_json)
class_names = {value:key for key,value in class_indices.items()}

In [ ]:
# Inspect number of samples per class
for class_name, class_index in class_indices.items():
    print(f'{class_name:25} {class_index:3} {np.count_nonzero(y==class_index)} samples')

In [ ]:
# Load class hierarchy
with open(class_hierarchy_json,'r') as file:
    class_hierarchy = json.load(file)
print(class_hierarchy)

In [ ]:
# Define grouped classes
grouped_classes = ['Sand','Bedrock','Cobble','Maerl','Rockweed','Kelp','Chorda filum']
grouped_class_indices = annotation.class_indices_from_hierarchy(class_hierarchy,class_indices,grouped_classes)
grouped_class_indices['Brown algae (deep)'] = {class_indices['Brown algae']}
print(grouped_class_indices)

In [ ]:
# Create "grouped" y vector
y_grouped = np.zeros_like(y)
for i, group_name in enumerate(grouped_class_indices.keys()):
    for ci in grouped_class_indices[group_name]:
        y_grouped[y==ci] = i+1

In [ ]:
# Class histogram
y_grouped_values = np.unique(y_grouped)
sample_count = [np.count_nonzero(y_grouped==ygv) for ygv in y_grouped_values]
plt.bar(y_grouped_values,sample_count)
bar_tick_labels = ['Not in groups']+[k for k in grouped_class_indices.keys()]
plt.xticks(y_grouped_values,bar_tick_labels,rotation=45)
plt.ylabel('Number of samples per class')
plt.show()

In [ ]:
print('Not included in any groups:')
for not_in_group_ind in np.unique(y[y_grouped==0]):
    print(f'  - {class_names[not_in_group_ind]} ({np.count_nonzero(y==not_in_group_ind)} samples)')

In [ ]:
# Limit to wavelength range and convert to logarithmic spectra
wl_ind = (wl>= wl_min) & (wl<= wl_max)
X_log = np.log10(X[:,wl_ind])
print(X_log.shape)

In [ ]:
# Create PCA model
pca = sklearn.decomposition.PCA(n_components=30)
_ = pca.fit(X_log)

In [ ]:
# Plot explained variance ratio
plt.semilogy(np.arange(1,pca.n_components+1),pca.explained_variance_ratio_,'-o')
plt.ylabel('Explained variance ratio')
plt.xlabel('Component number')
plt.show()

In [ ]:
print(f'Explained variance using 5 components (in log-domain): {np.sum(pca.explained_variance_ratio_[0:5])*100:.2f} %')

In [ ]:
fig, ax = plt.subplots(nrows=3,ncols=3,figsize=(12,12))
for i,component in enumerate(pca.components_[0:9]):
    ax[i//3][i%3].plot(wl[wl_ind],component)
    ax[i//3][i%3].title.set_text(f'Component {i+1}')

In [ ]:
plt.plot(wl[wl_ind],pca.mean_)
plt.show()

In [ ]:
# Save model
hyspec_ml.save_pca_model(pca,X_log,npz_filename=pca_model_path,n_components=5)
# Save wavelengths
wl_savename = pca_model_path.parent / (pca_model_path.stem + '_wl.npz')
np.savez(npz_filename)

In [ ]:
# Load saved model (check that saving was successful)
# W_pca,X_mean_loaded,X_std_loaded = hyspec_ml.read_pca_model(pca_model_path)

In [ ]:
# assert np.allclose(W_pca,np.transpose(pca.components_[0:8,:]),rtol=1e-2)  # Almost equal - PCA not always the same?
# assert np.allclose(X_mean, X_mean_loaded)
# assert np.allclose(X_std, X_std_loaded)